# Visualising the CirCONSTANCEs
## Processing and visualisation of network data in the Constance de Salm correspondence.

In [5]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import networkx as nx
import plotly.graph_objs as go

import pandas as pd
from colour import Color
from textwrap import dedent as d

In [6]:

# import the css template, and pass the css template into dash
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "CdS Correspondence Network – Interactive Version"

# Default parameters for beginning.
DECADE = [1790, 1840]
NAME = 'Salm, Constance de'

In [7]:
def network_graph(decade_range: list, name: str) -> None:
    """
    Function that is used to extract data from the dataframe according to the selected value range (e.g. decade range).
    :param decade_range: List with start and end value of a range (in this case, of the given decades).
    :param name: Name of the person whose node is to be centered.
    :return: None.
    """
    network_data = pd.read_csv('../data/retrieved/network_data.csv')[:]

    # Read data from Pandas Dataframe.
    correspondence_set = set()

    # Filtering according to input: Remove all datapoints that are not included in the decade range.
    if decade_range[0] == decade_range[1]:
        for index in range(0, len(network_data)):
            if network_data['decade'][index] != decade_range[0]:
                network_data.drop(axis=0, index=index, inplace=True)
                continue
            correspondence_set.add(network_data['Verfasser'][index])
            correspondence_set.add(network_data['Empfänger'][index])
    else:
        for index in range(0, len(network_data)):
            if decade_range[0] > network_data['decade'][index] or network_data['decade'][index] > decade_range[1]:
                network_data.drop(axis=0, index=index, inplace=True)
                continue
            correspondence_set.add(network_data['Verfasser'][index])
            correspondence_set.add(network_data['Empfänger'][index])

    # For node attributes – names as label.
    node_attr = {}
    for person in correspondence_set:
        temp_dict = {'name': person}
        node_attr[person] = temp_dict

    # Centric point will be chosen depending on the name that was entered beforehand.
    shells = []
    shell1 = [name]
    shells.append(shell1)
    shell2 = []
    for ele in correspondence_set:
        if ele != name:
            shell2.append(ele)
    shells.append(shell2)

    # Creating the network from the filtered DataFrame.
    G = nx.from_pandas_edgelist(network_data, 'Verfasser', 'Empfänger', ['Verfasser', 'decade', 'Empfänger', 'counts'],
                                create_using=nx.MultiDiGraph())
    nx.set_node_attributes(G, node_attr)
    if len(shell2) > 1:
        pos = nx.drawing.layout.shell_layout(G, shells)
    else:
        pos = nx.drawing.layout.spring_layout(G)
    for node in G.nodes:
        G.nodes[node]['pos'] = list(pos[node])

    if len(shell2) == 0:
        traceRecode = []  # contains edge_trace, node_trace, middle_node_trace

        node_trace = go.Scatter(x=tuple([1]), y=tuple([1]), text=tuple([str(name)]),
                                textposition="bottom center",
                                mode='markers+text',
                                marker={'size': 50, 'color': '#D89C7C'})
        traceRecode.append(node_trace)

        node_trace1 = go.Scatter(x=tuple([1]), y=tuple([1]),
                                 mode='markers',
                                 marker={'size': 50, 'color': '#D89C7C'},
                                 opacity=0)
        traceRecode.append(node_trace1)

        figure = {
            "data": traceRecode,
            "layout": go.Layout(title='Interactive Correspondence Visualisation', showlegend=False,
                                margin={'b': 40, 'l': 40, 'r': 40, 't': 40},
                                xaxis={'showgrid': False, 'zeroline': False, 'showticklabels': False},
                                yaxis={'showgrid': False, 'zeroline': False, 'showticklabels': False},
                                height=600
                                )}
        return figure

    traceRecode = []  # contains edge_trace, node_trace, middle_node_trace

    # Adding colour index for edge representation.
    colors = list(Color('#F1B62F').range_to(Color('#6D5214'), len(G.edges())))
    colors = ['rgb' + str(x.rgb) for x in colors]

    index = 0
    for edge in G.edges:
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        weight = float(G.edges[edge]['counts']) / max(network_data['counts']) * 10
        trace = go.Scatter(x=tuple([x0, x1, None]), y=tuple([y0, y1, None]),
                           mode='lines',
                           line={'width': weight},
                           marker=dict(color=colors[index]),
                           opacity=1)
        traceRecode.append(trace)
        index = index + 1

    # Defining trace on the nodes.
    node_trace = go.Scatter(x=[], y=[], hovertext=[], text=[], mode='markers+text', textposition="bottom center",
                            hoverinfo="text", marker={'size': 50, 'color': '#EF5F0D'})

    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        hovertext = "Name: " + str(G.nodes[node]['name'])
        text = G.nodes[node]['name']
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        node_trace['hovertext'] += tuple([hovertext])
        node_trace['text'] += tuple([text])

    traceRecode.append(node_trace)

    # Hover trace that will be shown when the user hovers over the edges or nodes; visualises text information about the datapoint.
    middle_hover_trace = go.Scatter(x=[], y=[], hovertext=[], mode='markers', hoverinfo="text",
                                    marker={'size': 20, 'color': '#C58560'},
                                    opacity=0)

    index = 0
    for edge in G.edges:
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        hovertext = "From: " + str(G.edges[edge]['Verfasser']) + "<br>" + "To: " + str(
            G.edges[edge]['Empfänger']) + "<br>" + "counts: " + str(
            G.edges[edge]['counts']) + "<br>" + "Decade: " + str(G.edges[edge]['decade'])
        middle_hover_trace['x'] += tuple([(x0 + x1) / 2])
        middle_hover_trace['y'] += tuple([(y0 + y1) / 2])
        middle_hover_trace['hovertext'] += tuple([hovertext])
        index = index + 1

    traceRecode.append(middle_hover_trace)

    # Creating figure that will later be added to the dash HTML layout.
    figure = {
        "data": traceRecode,
        "layout": go.Layout(title='Interactive Correspondence Visualisation', showlegend=False, hovermode='closest',
                            margin={'b': 40, 'l': 40, 'r': 40, 't': 40},
                            xaxis={'showgrid': False, 'zeroline': False, 'showticklabels': False},
                            yaxis={'showgrid': False, 'zeroline': False, 'showticklabels': False},
                            height=600,
                            clickmode='event+select',
                            annotations=[
                                dict(
                                    ax=(G.nodes[edge[0]]['pos'][0] + G.nodes[edge[1]]['pos'][0]) / 2,
                                    ay=(G.nodes[edge[0]]['pos'][1] + G.nodes[edge[1]]['pos'][1]) / 2, axref='x',
                                    ayref='y',
                                    x=(G.nodes[edge[1]]['pos'][0] * 3 + G.nodes[edge[0]]['pos'][0]) / 4,
                                    y=(G.nodes[edge[1]]['pos'][1] * 3 + G.nodes[edge[0]]['pos'][1]) / 4, xref='x',
                                    yref='y',
                                    opacity=1
                                ) for edge in G.edges]
                            )}
    return figure


In [ ]:
app.layout = html.Div([
    # Setting the title of the figure.
    html.Div([html.H1("Correspondence Network Graph")],
             className="row",
             style={'textAlign': "center"}),
    # Defining the row.
    html.Div(
        className="row",
        children=[
            # Adding left side components for year slider and name input field.
            html.Div(
                className="two columns",
                children=[
                    dcc.Markdown(d("""
                            **Time Range To Visualise**
                            Slide the bar to define decade range. Singular values are also possible.
                            """)),
                    html.Div(
                        className="twelve columns",
                        children=[
                            dcc.RangeSlider(
                                id='my-range-slider',
                                min=1790,
                                max=1840,
                                step=10,
                                value=[1790, 1840],
                                marks={
                                    1790: {'label': '1790'},
                                    1800: {'label': '1800'},
                                    1810: {'label': '1810'},
                                    1820: {'label': '1820'},
                                    1830: {'label': '1830'},
                                    1840: {'label': '1840'},
                                }
                            ),
                            html.Br(),
                            html.Div(id='output-container-range-slider')
                        ],
                        style={'height': '300px',
                               'color': '#DE690D'}
                    ),
                    html.Div(
                        className="twelve columns",
                        children=[
                            dcc.Markdown(d("""
                            **Search for a Person in the Correspondence**
                            Type the name to visualise.
                            """)),
                            dcc.Input(id="input1", type="text", placeholder="Salm, Constance de"),
                            html.Div(id="output")
                        ],
                        style={'height': '300px'}
                    )
                ]
            ),

            # Add middle graph component.
            html.Div(
                className="eight columns",
                children=[dcc.Graph(id="my-graph",
                                    figure=network_graph(DECADE, NAME))],
            ),
        ]
    )
])


# Adding callback for left side components.
@app.callback(
    dash.dependencies.Output('my-graph', 'figure'),
    [dash.dependencies.Input('my-range-slider', 'value'), dash.dependencies.Input('input1', 'value')])
def update_output(value, input1):
    DECADE = value
    NAME = input1
    return network_graph(value, input1)
    # to update the global variable of DECADE and NAME and choose data accordingly.

if __name__ == '__main__':
    app.run_server(port=8051)